# Introduction

The purpose of this project is to analyse supermarket sales data through asking Key business questions such as 
'What product categories are performing well and not well at generating profits?' and 'Which of our branches are not performing well in Total sales and customer satisfaction?', then writing SQL queries to retrieve the data that answers those questions.

## Dataset metadata 

Attribute descriptions

Invoice id: Computer generated sales slip invoice identification number

Branch: Branch of supercenter (3 branches are available identified by A, B and C).

City: Location of supercenters

Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.

Gender: Gender type of customer

Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel

Unit price: Price of each product in $

Quantity: Number of products purchased by customer

Tax: 5% tax fee for customer buying

Total: Total price including tax

Date: Date of purchase (Record available from January 2019 to March 2019)

Time: Purchase time (10am to 9pm)

Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)

COGS: Cost of goods sold

Gross margin percentage: Gross margin percentage

Gross income: Gross income

Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql.functions import col, unix_timestamp, to_date, to_timestamp, to_str, concat_ws

In [2]:
spark = pyspark.sql.SparkSession.builder.appName("Sales queries").getOrCreate()

In [3]:
spark

In [4]:
df_raw = spark.read.csv('data/supermarket_sales.csv', header=True, sep=",").cache()
print('Total Records = {}'.format(df_raw.count()))
df_raw.printSchema()

Total Records = 1000
root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Tax 5%: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: string (nullable = true)
 |-- gross margin percentage: string (nullable = true)
 |-- gross income: string (nullable = true)
 |-- Rating: string (nullable = true)



In [5]:
df_raw.show(5)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date| Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|13:08|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|10:29|       Cash|  76.4|            4.761904762|        3.82|   9.6|
|631-41-3108|     A|   Yangon|       Normal|  Male|  Ho

## Convert Date and Time columns to proper format

In [6]:
dir(pyspark.sql.functions)

['Column',
 'DataFrame',
 'DataType',
 'PandasUDFType',
 'PythonEvalType',
 'SparkContext',
 'StringType',
 'UserDefinedFunction',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_create_column_from_literal',
 '_create_lambda',
 '_create_udf',
 '_get_get_jvm_function',
 '_get_lambda_parameters',
 '_invoke_binary_math_function',
 '_invoke_function',
 '_invoke_function_over_column',
 '_invoke_higher_order_function',
 '_options_to_str',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_unresolved_named_lambda_variable',
 'abs',
 'acos',
 'acosh',
 'add_months',
 'aggregate',
 'approxCountDistinct',
 'approx_count_distinct',
 'array',
 'array_contains',
 'array_distinct',
 'array_except',
 'array_intersect',
 'array_join',
 'array_max',
 'array_min',
 'array_position',
 'array_remove',
 'array_repeat',
 'array_sort',
 'array_union',
 'arrays_overlap',
 'arrays_zip',
 'asc',
 'asc_nulls_first',
 'asc_nulls_last',
 'ascii',
 'asi

In [7]:
# Convert Time column to timestamp after concatenating it with date
# or else the default system date of 1970-01-01 will be used for the day
df = df_raw.withColumn('Time', concat_ws('_','Date','Time'))
df = df.withColumn('Time', 
                  to_timestamp(unix_timestamp(col('Time'), 'M/d/yyyy_HH:mm').cast("timestamp")))

# Convert Date column to timestamp
df = df.withColumn('Date', 
                  to_date(unix_timestamp(col('Date'), 'M/d/yyyy').cast("timestamp")))

df.printSchema()
df.show(5)

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Tax 5%: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: string (nullable = true)
 |-- gross margin percentage: string (nullable = true)
 |-- gross income: string (nullable = true)
 |-- Rating: string (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   To

In [8]:
# Creating a function for easy display of SQL query pyspark dataframes
df.registerTempTable("sales")

def show_df(sqlquery):
    spark.sql(sqlquery).show()

## General

In [9]:
# Sales per product line
show_df("""SELECT `Product line`,count(*) as count
            FROM sales
            GROUP BY `Product line`""")

+--------------------+-----+
|        Product line|count|
+--------------------+-----+
|  Home and lifestyle|  160|
| Fashion accessories|  178|
|   Health and beauty|  152|
|Electronic access...|  170|
|  Food and beverages|  174|
|   Sports and travel|  166|
+--------------------+-----+



In [10]:
# Total sales per product line
show_df("""SELECT `Product line`, round(sum(`total`), 2) as total_sales
           FROM sales
           GROUP BY `Product line`""")

+--------------------+-----------+
|        Product line|total_sales|
+--------------------+-----------+
|  Home and lifestyle|   53861.91|
| Fashion accessories|    54305.9|
|   Health and beauty|   49193.74|
|Electronic access...|   54337.53|
|  Food and beverages|   56144.84|
|   Sports and travel|   55122.83|
+--------------------+-----------+



In [11]:
show_df('Select min(Date) as first_date,max(Date) as last_date from sales')

+----------+----------+
|first_date| last_date|
+----------+----------+
|2019-01-01|2019-03-30|
+----------+----------+



In [12]:
show_df('Select Branch, City from sales group by Branch, City order by Branch')

+------+---------+
|Branch|     City|
+------+---------+
|     A|   Yangon|
|     B| Mandalay|
|     C|Naypyitaw|
+------+---------+



In [13]:
# Average rating by Branch and gender
show_df('Select Branch, Gender, Round(avg(Rating),3) as Avg_rating from sales group by Branch, Gender order by Branch')

+------+------+----------+
|Branch|Gender|Avg_rating|
+------+------+----------+
|     A|  Male|     7.196|
|     A|Female|     6.839|
|     B|Female|     6.877|
|     B|  Male|     6.762|
|     C|  Male|     6.972|
|     C|Female|     7.158|
+------+------+----------+



## Select average gross margin percentage by product line, and total sales by product line and gender
What product lines are under and over performing in terms of creating profit?

In [14]:
show_df("""SELECT `Product line`, round(avg(`gross margin percentage`),2) as Avg_margin_percent, 
            round(sum(`Gross income`),2) as total_profit
           FROM sales
           GROUP BY `Product line`
           ORDER BY `Product line` ASC""")

+--------------------+------------------+------------+
|        Product line|Avg_margin_percent|total_profit|
+--------------------+------------------+------------+
|Electronic access...|              4.76|      2587.5|
| Fashion accessories|              4.76|      2586.0|
|  Food and beverages|              4.76|     2673.56|
|   Health and beauty|              4.76|     2342.56|
|  Home and lifestyle|              4.76|     2564.85|
|   Sports and travel|              4.76|      2624.9|
+--------------------+------------------+------------+



In [15]:
# Sales by product line and gender
show_df("""SELECT `Product line`, Gender, round(sum(`Total`)) as total_sales
           FROM sales
           GROUP BY `Product line`, Gender
           ORDER BY  `Product line` ASC, Gender ASC""")

+--------------------+------+-----------+
|        Product line|Gender|total_sales|
+--------------------+------+-----------+
|Electronic access...|Female|    27102.0|
|Electronic access...|  Male|    27236.0|
| Fashion accessories|Female|    30437.0|
| Fashion accessories|  Male|    23868.0|
|  Food and beverages|Female|    33171.0|
|  Food and beverages|  Male|    22974.0|
|   Health and beauty|Female|    18561.0|
|   Health and beauty|  Male|    30633.0|
|  Home and lifestyle|Female|    30037.0|
|  Home and lifestyle|  Male|    23825.0|
|   Sports and travel|Female|    28575.0|
|   Sports and travel|  Male|    26548.0|
+--------------------+------+-----------+



## Select Rating and Profit by Branch
What  branches are successfully satisfying customers and successfully generating profit?

In [16]:
output =  spark.sql("""SELECT Branch, Round(avg(rating),3) as Avg_rating, sum(`gross income`) as Total_profit
                        FROM sales
                        GROUP BY Branch
                        ORDER BY Branch ASC""")
output.show()

+------+----------+-----------------+
|Branch|Avg_rating|     Total_profit|
+------+----------+-----------------+
|     A|     7.027|5057.160500000002|
|     B|     6.818|5057.032000000003|
|     C|     7.073|5265.176500000002|
+------+----------+-----------------+



In [17]:
show_df("""WITH C as (SELECT Branch, sum(`Unit price` * Quantity) as Cogs
                             FROM sales
                             GROUP BY Branch),
       P as (SELECT Branch, sum(`gross income`) as Profit
                             FROM sales
                             GROUP BY Branch)
        SELECT C.BRANCH, C.Cogs as Cost_of_goods, P.profit, (C.cogs + P.Profit) as Revenue
        FROM C 
        JOIN P ON C.Branch = P.Branch
        ORDER BY C.BRANCH ASC
        """
)

+------+------------------+-----------------+------------------+
|BRANCH|     Cost_of_goods|           profit|           Revenue|
+------+------------------+-----------------+------------------+
|     A|101143.21000000006|5057.160500000002|106200.37050000006|
|     B|101140.63999999993|5057.032000000003|106197.67199999993|
|     C|         105303.53|5265.176500000002|       110568.7065|
+------+------------------+-----------------+------------------+



In [18]:
show_df("""SELECT Branch, COUNT(Distinct(`Invoice ID`)) as branch_sales
                                            FROM Sales 
                                            GROUP by Branch""")

+------+------------+
|Branch|branch_sales|
+------+------------+
|     B|         332|
|     C|         328|
|     A|         340|
+------+------------+



In [19]:
#Is there a preference for certain product lines at certain branches?

show_df("""With BS as (SELECT Branch, COUNT(Distinct(`Invoice ID`)) as branch_sales
                        FROM Sales 
                        GROUP by Branch  
                            ),

           PBS as (SELECT Branch, `Product line` as pl, Count(Distinct(`Invoice ID`)) as num_sales
                    FROM sales
                    GROUP BY Branch, `Product line`
                    ORDER BY Branch ASC, `Product line`)
                    
            SELECT PBS.Branch, PBS.pl as `Product line`, PBS.num_sales as `Number of sales`, 
            Round(PBS.num_sales/BS.branch_sales, 3) as `Percent of branch sales`
            FROM PBS join BS
            ON PBS.Branch = Bs.Branch
            ORDER BY Branch, `Product line`
                        """)

+------+--------------------+---------------+-----------------------+
|Branch|        Product line|Number of sales|Percent of branch sales|
+------+--------------------+---------------+-----------------------+
|     A|Electronic access...|             60|                  0.176|
|     A| Fashion accessories|             51|                   0.15|
|     A|  Food and beverages|             58|                  0.171|
|     A|   Health and beauty|             47|                  0.138|
|     A|  Home and lifestyle|             65|                  0.191|
|     A|   Sports and travel|             59|                  0.174|
|     B|Electronic access...|             55|                  0.166|
|     B| Fashion accessories|             62|                  0.187|
|     B|  Food and beverages|             50|                  0.151|
|     B|   Health and beauty|             53|                   0.16|
|     B|  Home and lifestyle|             50|                  0.151|
|     B|   Sports an

## Select by date

In [20]:
# Invoices in the first 2 days
show_df("""SELECT Date, `Invoice ID`
            FROM sales
            WHERE DATE_TRUNC('day', Date) = '2019-01-01' or DATE_TRUNC('day', Date) = '2019-01-02'
            SORT BY Date""")

+----------+-----------+
|      Date| Invoice ID|
+----------+-----------+
|2019-01-01|765-26-6951|
|2019-01-01|530-90-9855|
|2019-01-01|891-01-7034|
|2019-01-01|493-65-6248|
|2019-01-01|556-97-7101|
|2019-01-01|133-14-7229|
|2019-01-01|651-88-7328|
|2019-01-01|182-52-7000|
|2019-01-01|416-17-9926|
|2019-01-01|271-77-8740|
|2019-01-01|770-42-8960|
|2019-01-01|746-04-1077|
|2019-01-02|504-35-8843|
|2019-01-02|446-47-6729|
|2019-01-02|244-08-0162|
|2019-01-02|198-84-7132|
|2019-01-02|744-09-5786|
|2019-01-02|712-39-0363|
|2019-01-02|345-68-9016|
|2019-01-02|670-71-7306|
+----------+-----------+



In [21]:
# Monthly income by branch per month
show_df("""SELECT Branch, DATE_TRUNC('month', Date) as Month, Round(Sum(`Gross income`),2) as Gross_income 
            FROM sales
            GROUP BY Branch, Month
            ORDER BY Branch, Month
            """)

+------+-------------------+------------+
|Branch|              Month|Gross_income|
+------+-------------------+------------+
|     A|2019-01-01 00:00:00|     1841.96|
|     A|2019-02-01 00:00:00|     1421.91|
|     A|2019-03-01 00:00:00|     1793.29|
|     B|2019-01-01 00:00:00|     1770.29|
|     B|2019-02-01 00:00:00|     1639.25|
|     B|2019-03-01 00:00:00|     1647.49|
|     C|2019-01-01 00:00:00|     1925.46|
|     C|2019-02-01 00:00:00|     1568.33|
|     C|2019-03-01 00:00:00|     1771.38|
+------+-------------------+------------+



In [22]:
# Monthly income by branch for first 2 months
show_df("""SELECT Branch, DATE_TRUNC('month', Date) as Month, Round(Sum(`Gross income`),2) as Gross_income 
            FROM sales
            WHERE DATE_TRUNC('Month', Date) = '2019-01-01' or DATE_TRUNC('Month', Date) = '2019-02-01'
            GROUP BY Branch, Month
            ORDER BY Branch, Month
            """)

+------+-------------------+------------+
|Branch|              Month|Gross_income|
+------+-------------------+------------+
|     A|2019-01-01 00:00:00|     1841.96|
|     A|2019-02-01 00:00:00|     1421.91|
|     B|2019-01-01 00:00:00|     1770.29|
|     B|2019-02-01 00:00:00|     1639.25|
|     C|2019-01-01 00:00:00|     1925.46|
|     C|2019-02-01 00:00:00|     1568.33|
+------+-------------------+------------+



In [23]:
# What days of the week have the most and least sales
# Sunday is 1, Saturday is 7
show_df("""SELECT dayofweek(Date) as Day_of_week, Round(Sum(Total),2) as Total_sales, 
            Round(AVG(Total),4) as Average_sales_per_transaction
            FROM sales
            GROUP BY Day_of_week
            ORDER BY Day_of_week
            Limit 7
""")

+-----------+-----------+-----------------------------+
|Day_of_week|Total_sales|Average_sales_per_transaction|
+-----------+-----------+-----------------------------+
|          1|   44457.89|                     334.2699|
|          2|   37899.08|                     303.1926|
|          3|   51482.25|                      325.837|
|          4|   43731.14|                     305.8121|
|          5|   45349.25|                     328.6177|
|          6|   43926.34|                     316.0168|
|          7|   56120.81|                     342.2001|
+-----------+-----------+-----------------------------+



## Running total sales, Lagged sales, Sales delta/growth by week
### Analysing sales KPI by weeks

In [24]:
# Weekly sales count
show_df("""SELECT DATE_TRUNC('week', Date) as Week_start, COUNT(*) as Sales
            FROM sales
            GROUP BY Week_start
            ORDER BY Week_start ASC
            """)

+-------------------+-----+
|         Week_start|Sales|
+-------------------+-----+
|2018-12-31 00:00:00|   55|
|2019-01-07 00:00:00|   73|
|2019-01-14 00:00:00|   82|
|2019-01-21 00:00:00|   93|
|2019-01-28 00:00:00|   83|
|2019-02-04 00:00:00|   92|
|2019-02-11 00:00:00|   72|
|2019-02-18 00:00:00|   60|
|2019-02-25 00:00:00|   87|
|2019-03-04 00:00:00|   88|
|2019-03-11 00:00:00|   78|
|2019-03-18 00:00:00|   76|
|2019-03-25 00:00:00|   61|
+-------------------+-----+



In [25]:
# Weekly running sales
show_df("""WITH weekly_sales as(SELECT DATE_TRUNC('week', Date) as Week_start, COUNT(*) as sales
            FROM sales
            GROUP BY Week_start
            ORDER BY Week_start ASC)
            
            SELECT Week_start, sales, sum(sales) over (ORDER BY Week_start ASC) as Running_sales
            FROM weekly_sales
            ORDER BY Week_start ASC
            """)

+-------------------+-----+-------------+
|         Week_start|sales|Running_sales|
+-------------------+-----+-------------+
|2018-12-31 00:00:00|   55|           55|
|2019-01-07 00:00:00|   73|          128|
|2019-01-14 00:00:00|   82|          210|
|2019-01-21 00:00:00|   93|          303|
|2019-01-28 00:00:00|   83|          386|
|2019-02-04 00:00:00|   92|          478|
|2019-02-11 00:00:00|   72|          550|
|2019-02-18 00:00:00|   60|          610|
|2019-02-25 00:00:00|   87|          697|
|2019-03-04 00:00:00|   88|          785|
|2019-03-11 00:00:00|   78|          863|
|2019-03-18 00:00:00|   76|          939|
|2019-03-25 00:00:00|   61|         1000|
+-------------------+-----+-------------+



In [26]:
# Weekly Lagged sales
show_df("""WITH weekly_sales as(SELECT DATE_TRUNC('week', Date) as Week_start, COUNT(*) as sales
            FROM sales
            GROUP BY Week_start
            ORDER BY Week_start ASC)
            
            SELECT Week_start, sales, coalesce(lag(sales) over (ORDER BY Week_start ASC), 0) as Lagged_sales
            FROM weekly_sales
            ORDER BY Week_start ASC
            """)

+-------------------+-----+------------+
|         Week_start|sales|Lagged_sales|
+-------------------+-----+------------+
|2018-12-31 00:00:00|   55|           0|
|2019-01-07 00:00:00|   73|          55|
|2019-01-14 00:00:00|   82|          73|
|2019-01-21 00:00:00|   93|          82|
|2019-01-28 00:00:00|   83|          93|
|2019-02-04 00:00:00|   92|          83|
|2019-02-11 00:00:00|   72|          92|
|2019-02-18 00:00:00|   60|          72|
|2019-02-25 00:00:00|   87|          60|
|2019-03-04 00:00:00|   88|          87|
|2019-03-11 00:00:00|   78|          88|
|2019-03-18 00:00:00|   76|          78|
|2019-03-25 00:00:00|   61|          76|
+-------------------+-----+------------+



In [27]:
# Changes in number of sales every week and changes in sales growth (Percent)
# ASSUME THAT THERE WERE 50 SALES IN THE WEEK PRIOR TO THE WEEK THAT BEGAN ON 2019-12-31

show_df("""WITH weekly_sales as(SELECT DATE_TRUNC('week', Date) as Week_start, COUNT(*) as Sales
            FROM sales
            GROUP BY Week_start
            ORDER BY Week_start ASC),
            
            weekly_sales_lag as(SELECT Week_start, sales, coalesce(lag(sales) over (ORDER BY Week_start ASC), 50) as Lagged_sales
            FROM weekly_sales
            ORDER BY Week_start ASC)
            
            SELECT Week_start, Sales-Lagged_sales as Sales_delta, round((Sales-Lagged_sales)/Lagged_sales,2) as Sales_growth
            FROM weekly_sales_lag
            ORDER BY Week_start ASC
            """)

+-------------------+-----------+------------+
|         Week_start|Sales_delta|Sales_growth|
+-------------------+-----------+------------+
|2018-12-31 00:00:00|          5|         0.1|
|2019-01-07 00:00:00|         18|        0.33|
|2019-01-14 00:00:00|          9|        0.12|
|2019-01-21 00:00:00|         11|        0.13|
|2019-01-28 00:00:00|        -10|       -0.11|
|2019-02-04 00:00:00|          9|        0.11|
|2019-02-11 00:00:00|        -20|       -0.22|
|2019-02-18 00:00:00|        -12|       -0.17|
|2019-02-25 00:00:00|         27|        0.45|
|2019-03-04 00:00:00|          1|        0.01|
|2019-03-11 00:00:00|        -10|       -0.11|
|2019-03-18 00:00:00|         -2|       -0.03|
|2019-03-25 00:00:00|        -15|        -0.2|
+-------------------+-----------+------------+



In [28]:
# Changes in number of sales every week and changes in sales growth (Percent)
# Assume Income in prior week was 10000

show_df("""WITH weekly_sales as(SELECT DATE_TRUNC('week', Date) as Week_start, SUM(Total) as Sales
            FROM sales
            GROUP BY Week_start
            ORDER BY Week_start ASC),
            -- Assume income in prior week
            weekly_sales_lag as(SELECT Week_start, Sales, coalesce(lag(Sales) over (ORDER BY Week_start ASC), 10000) as Lagged_sales
            FROM weekly_sales
            ORDER BY Week_start ASC)
            
            SELECT Week_start, Round(Sales-Lagged_sales,2) as Revenue_delta, round((Sales-Lagged_sales)/Lagged_sales,2) as Revenue_growth
            FROM weekly_sales_lag
            ORDER BY Week_start ASC
            """)

+-------------------+-------------+--------------+
|         Week_start|Revenue_delta|Revenue_growth|
+-------------------+-------------+--------------+
|2018-12-31 00:00:00|      7543.39|          0.75|
|2019-01-07 00:00:00|      6917.81|          0.39|
|2019-01-14 00:00:00|      4232.16|          0.17|
|2019-01-21 00:00:00|       593.52|          0.02|
|2019-01-28 00:00:00|      -926.44|         -0.03|
|2019-02-04 00:00:00|     -1258.61|         -0.04|
|2019-02-11 00:00:00|     -1538.24|         -0.06|
|2019-02-18 00:00:00|     -8234.93|         -0.32|
|2019-02-25 00:00:00|     11891.05|          0.69|
|2019-03-04 00:00:00|      -800.88|         -0.03|
|2019-03-11 00:00:00|     -4428.62|         -0.16|
|2019-03-18 00:00:00|      1130.38|          0.05|
|2019-03-25 00:00:00|      -7242.6|         -0.29|
+-------------------+-------------+--------------+

